In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator itrymport get_image_uri
from sagemaker.session import s3_input, Session

In [11]:
bucketName = 'get2jawa-2021'
myRegion = boto3.session.Session().region_name
print(myRegion)

us-east-2


In [12]:
s3 = boto3.resource('s3')
try:
    if myRegion == 'us-east-2':
        s3.create_bucket(Bucket=bucketName, CreateBucketConfiguration={'LocationConstraint':'us-east-2'})
    print("S3 bucket created successfully")
except Exception as ex:
    print("s3 error : ", ex)

S3 bucket created successfully


In [14]:
prefix = 'xgboost-as-builtin-algo'
output_path = 's3://{}/{}/output'.format(bucketName, prefix)
print(output_path)

s3://get2jawa-2021/xgboost-as-builtin-algo/output


## Downloading dataset and storing it in S3

In [15]:
import pandas as pd
import urllib

In [16]:
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "sample.csv")
    print("success : Downloaded dataset")
except Exception as e:
    print("Error occured  : ", e)

success : Downloaded dataset


In [17]:
try:
    model_data = pd.read_csv('sample.csv')
    print('Dataframe loaded')
except Exeption as e:
    print('Data load error : ', e)

Dataframe loaded


In [18]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 62) (12357, 62)


In [20]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketName).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucketName, prefix), content_type='csv')

In [21]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketName).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucketName, prefix), content_type='csv')

In [27]:
container = sagemaker.image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.0-1')

In [28]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [37]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [38]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2021-05-25 01:02:24 Starting - Starting the training job...
2021-05-25 01:02:47 Starting - Launching requested ML instancesProfilerReport-1621904543: InProgress
......
2021-05-25 01:03:47 Starting - Preparing the instances for training.........
2021-05-25 01:05:22 Downloading - Downloading input data
2021-05-25 01:05:22 Training - Training image download completed. Training in progress.
2021-05-25 01:05:22 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[01:05:19] 28831x60 matr

In [39]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-------------!

In [41]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucketName)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': 'XXE3H3DK4TWN008T',
   'HostId': 'JygWMoay/ggjSR+Ot3yqQGxdgmSwixvqaPDkPaHtdUCIXd9ZPBQrtBfrlDEVg5IqzQwekGEPbLU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'JygWMoay/ggjSR+Ot3yqQGxdgmSwixvqaPDkPaHtdUCIXd9ZPBQrtBfrlDEVg5IqzQwekGEPbLU=',
    'x-amz-request-id': 'XXE3H3DK4TWN008T',
    'date': 'Tue, 25 May 2021 05:37:14 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-builtin-algo/output/sagemaker-xgboost-2021-05-25-01-02-23-908/output/model.tar.gz'},
   {'Key': 'xgboost-as-builtin-algo/output/sagemaker-xgboost-2021-05-25-01-02-23-908/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost-as-builtin-algo/output/sagemaker-xgboost-2021-05-25-01-02-23-908/rule-output/ProfilerReport-1621904543/profiler-output/profiler-reports/Dataloader.json'},
   {'Key': 'xgboost-as-builtin-algo/output